In [5]:
import numpy as np
data = "35.042543, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, 35.062648, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, 35.082654, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, 35.102551, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, 35.12204, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, 35.142053, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, 35.161749, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, 35.182654, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, ,"
data = data.split(", ")
# data = data[0:-1]
# print(data)
data_size = 10
data = np.array([float(s) for s in data[0:-1]])
# for s in data:
#     print(s)
#     float(s.replace(",",""))
data = (np.reshape(data, (-1, data_size))).tolist()
num = data[-1][0] - data[0][0]
print(data)
print(num)
#print(data_final)

[[35.042543, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [35.062648, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [35.082654, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [35.102551, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [35.12204, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [35.142053, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [35.161749, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [35.182654, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
0.14011099999999743


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm

DATADIR = "X:/Datasets/PetImages"

CATEGORIES = ["Dog", "Cat"]

for category in CATEGORIES:  # do dogs and cats
    path = os.path.join(DATADIR,category)  # create path to dogs and cats
    for img in os.listdir(path):  # iterate over each image per dogs and cats
        img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
        plt.imshow(img_array, cmap='gray')  # graph it
        plt.show()  # display!

        break  # we just want one for now so break
    break  #...and one more!

IMG_SIZE = 100

new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
plt.imshow(new_array, cmap='gray')
plt.show()

In [ ]:
import random
training_data = []

def create_training_data():
    for category in CATEGORIES:  # do dogs and cats

        path = os.path.join(DATADIR,category)  # create path to dogs and cats
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=dog 1=cat

        for img in tqdm(os.listdir(path)):  # iterate over each image per dogs and cats
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                training_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass
            #except OSError as e:
            #    print("OSErrroBad img most likely", e, os.path.join(path,img))
            #except Exception as e:
            #    print("general exception", e, os.path.join(path,img))

create_training_data()

print(len(training_data))


random.shuffle(training_data)

In [ ]:
import pickle
X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)

print(X[0].reshape(-1, IMG_SIZE, IMG_SIZE, 1))

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)


pickle_out = open("X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
# more info on callbakcs: https://keras.io/callbacks/ model saver is cool too.
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time

pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0

dense_layers = [0]
layer_sizes = [64]
conv_layers = [3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)

            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())

            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

            model.compile(loss='binary_crossentropy',
                          optimizer='adam',
                          metrics=['accuracy'],
                          )

            model.fit(X, y,
                      batch_size=32,
                      epochs=10,
                      validation_split=0.3,
                      callbacks=[tensorboard])

model.save('64x3-CNN.model')

FileNotFoundError: [Errno 2] No such file or directory: 'X.pickle'

In [ ]:
import cv2
import tensorflow as tf


def getfile(filepath):
    IMG_SIZE = 70
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    
    return

model = tf.keras.models.load_model("")
prediction = model.predict()
print(prediction)

In [28]:
from googlesearch import search 
import time
# google keywords to search
query = ["diy ", "reuse "]

def search_diy(material: str, results: int = 4, waittime: int = 2):
    global query
    
    #appends the material to the keywords
    terms = [(s + material) for s in query]
    
    queries_per_search = results/len(query)
    
    # quick check
    if queries_per_search < 0:
        raise ValueError("This will yield no search results")
        
    search_results = []
    for i in terms:
        for j in search(i, tld='com', lang='en', num= results, stop= results, pause=waittime): 
            search_results.append(j)
    return search_results




search_diy(5)


TypeError: can only concatenate str (not "int") to str

In [5]:
from googlesearch import search

In [57]:
import pandas as pd
import random
import numpy as np

starting_coords = {"longitude": -96.79,
                   "latitude" : 32.77
                  }

num_samples = 500
latitude_list = []
longitude_list = []
point_list = []

d = { "Longitude" : [],
     "Latitude" : [],
     "Points" : []
}
for i in range (num_samples):
    d["Longitude"].append(round(random.uniform(-101, -95), 2))
    d["Latitude"].append(round(random.uniform(31,34), 2))
    d["Points"].append(random.randint(1, 100))

print(d)
df = pd.DataFrame(d)

#print(df)
df.to_csv("test.csv", index = False)

{'Longitude': [-97.03, -98.84, -99.79, -95.47, -98.43, -95.47, -95.65, -100.6, -97.12, -100.14, -99.12, -97.51, -99.45, -96.1, -95.4, -96.21, -96.89, -98.53, -95.4, -98.27, -95.67, -100.67, -95.45, -97.26, -95.09, -97.19, -95.55, -100.72, -95.79, -100.35, -95.3, -97.69, -97.05, -97.22, -98.55, -99.42, -98.16, -97.97, -96.23, -100.51, -98.18, -97.57, -95.92, -95.1, -99.57, -98.41, -95.6, -100.6, -97.22, -99.92, -100.38, -95.04, -100.35, -96.73, -95.96, -97.87, -99.14, -95.0, -99.1, -98.26, -100.73, -95.51, -96.46, -97.11, -98.63, -97.41, -99.04, -98.09, -97.33, -100.12, -99.12, -96.72, -99.49, -99.22, -95.26, -100.78, -100.57, -95.89, -98.26, -99.93, -96.5, -100.17, -97.56, -96.1, -97.95, -96.29, -96.44, -97.99, -98.48, -99.03, -97.0, -98.92, -95.86, -96.06, -96.41, -97.96, -98.12, -96.16, -100.92, -100.63, -97.87, -100.64, -96.75, -96.4, -96.81, -95.35, -96.41, -98.19, -99.68, -96.57, -100.88, -97.12, -100.29, -96.69, -95.72, -98.54, -95.82, -96.57, -96.18, -98.15, -95.09, -98.97, -95.

In [59]:
d = {"Longitude" : [-97.33, -96.79, -96.82],
                   "Latitude" : [32.76, 32.78, 33.15],
                    "Points" : [750, 500, 1000]
                  }
print(d)
df = pd.DataFrame(d)
df.to_csv("test2.csv", index = False)

{'Longitude': [-97.33, -96.79, -96.82], 'Latitude': [32.76, 32.78, 33.15], 'Points': [750, 500, 1000]}
